In [1]:
import numpy as np
# import cv2
import tifffile as tiff
from shapely.geometry import Polygon, mapping, shape, box
from skimage import io
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
# import json
from rasterio import features
import scanpy as sc
import pandas as pd
import geopandas as gpd
%matplotlib inline
import pickle
import scipy

import seaborn as sns

from SGanalysis.SGobject import SGobject

In [2]:
filepath  = '/Users/grantkinsler/Dropbox (RajLab)/Shared_Grant/SpatialBarcodes/ImagingData/2024-02-27_spatialbarcodes_SG_expression/projects/2024-02-27_spatialbarcodes_expression/'
roi_name = 'roi_2'

In [ ]:
## import segmentation

from SGanalysis.SGobject import SGobject

# Create an instance of SGobject
sg_obj = SGobject()

# Convert a TIFF image to polygons and store them in a GeoDataFrame
print("Running mask_to_objects...")
sg_obj.mask_to_objects(f'{filepath}{roi_name}/segmentations/segmentation_1_nuclei.tiff')

In [ ]:
## import spots

spot_file = f'{filepath}{roi_name}/exports/decode_20240314_withcoordinates.csv'

sg_obj.load_points(spot_file)

In [ ]:
## associate spots with segmentation

sg_obj.create_cell_gene_table()

In [3]:
## save the sg object to a pickle file so that we don't have to redo all of the above steps that take a while in the future.

# with open(f'{filepath}{roi_name}/exports/sg_object_decode_20240314_segmentation_1.pkl', 'wb') as f:
#     pickle.dump(sg_obj, f)

## if we wanted to load the pickle file, we could run the commented code below

with open(f'{filepath}{roi_name}/exports/sg_object_decode_20240314_segmentation_1.pkl', 'rb') as f:
    sg_obj = pickle.load(f)

In [6]:
sg_obj.gdf.centroid

0             POINT (9310.387 8.913)
1            POINT (6475.859 12.348)
2           POINT (24695.071 14.397)
3           POINT (11512.465 18.850)
4           POINT (14620.955 16.886)
                    ...             
16371    POINT (19899.601 35594.028)
16372    POINT (20379.341 35601.878)
16373    POINT (22879.829 35591.004)
16374    POINT (24816.056 35588.584)
16375    POINT (12979.797 35592.946)
Length: 16376, dtype: geometry

In [ ]:
# get the cell by gene table
df = sg_obj.get_cell_gene_table_df()

In [ ]:
# look at the first few rows of the dataframe
df.head()

In [ ]:
# let's print some basic statistics about the data
sg_obj.generate_statistics_and_histograms()

Goal 1. Cluster cells based on their gene expression from the spatial genomics data.

In [ ]:
# associate with the barcode information


cell_barcodes = {}

# df.sort_index(inplace=True)

# set cutoff for barcode counting in analysis

cutoff = 1 # need more than this number of spots to call a barocde.
# cutoff = 3

n_cells = {}

for bc in ['bc_{:03d}'.format(i) for i in range(1,97)]:
    n_cells[bc] = len(df[df[bc]>cutoff][bc].values)


df['cell_id'] = df.index

for cell_id in df['cell_id']:
    this_cell = df[df['cell_id']==cell_id]
    cell_barcodes[cell_id] = []

    for bc in barcode_cols:
        if this_cell[bc].values[0] > cutoff:
            cell_barcodes[cell_id].append(bc)

df['called_barcodes'] = cell_barcodes.values()
df['n_called_barcodes'] = [len(bc_set) for bc_set in cell_barcodes.values()]
df['barcode_names'] = ['-'.join(sorted(bc_set)) for bc_set in cell_barcodes.values()]

# if working with sg_obj, this merges and gives you cell locations.
# df = pd.merge(df,sg_obj.gdf[['nucleus_centroid','nucleus']],how='left',left_index=True,right_index=True)

# # get each cell polygon's position
# df['location'] = sg_obj.gdf['nucleus'].centroid.values
# df['location_x'] = sg_obj.gdf['nucleus'].centroid.x.values
# df['location_y'] = sg_obj.gdf['nucleus'].centroid.y.values